In [1]:
!pip install google-generativeai==0.8.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.0/165.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.4/725.4 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.6
    Uninstalling google-ai-generativelanguage-0.6.6:
      Successfully uninstalled google-ai-generativelanguage-0.6.6
  Attempting uninstall: google-generativeai
    Found existing installation: google-generativeai 0.7.2
    Uninstalling google-generativeai-0.7.2:
      Successfully uninstalled google-generativeai-0.7.2


In [1]:
# @title 기본 사용법 1 – 싱글턴으로 메시지 주고받기
import google.generativeai as genai
import os
from google.colab import userdata
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))

In [2]:
model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content("인공지능에 대해 한 문장으로 설명하세요.")
print(response.text)

인공지능은 컴퓨터가 인간과 같은 지능적 행동을 수행하도록 훈련하는 것을 목표로 하는 컴퓨터 과학의 한 분야입니다. 



In [3]:
# @title 기본 사용법 2 – 멀티턴으로 메시지 주고받기(1)
from google.generativeai import ChatSession
model = genai.GenerativeModel('gemini-1.5-flash')
chat_session: ChatSession = model.start_chat(history=[])  # ChatSession 객체 반환
user_queries = ["인공지능에 대해 한 문장으로 짧게 설명하세요.", "의식이 있는지 한 문장으로 답하세요."]
for user_query in user_queries:
    print(f'[사용자]: {user_query}')
    response = chat_session.send_message(user_query)
    print(f'[모델]: {response.text}')

[사용자]: 인공지능에 대해 한 문장으로 짧게 설명하세요.
[모델]: 인공지능(AI)은 인간의 지능을 모방하여 복잡한 문제를 해결하고 인간과 같은 방식으로 학습하고 결정을 내리는 컴퓨터 시스템을 말합니다. 

[사용자]: 의식이 있는지 한 문장으로 답하세요.
[모델]: 저는 의식이 없습니다. 저는 Google에서 훈련된 대규모 언어 모델입니다. 



In [4]:
# @title 기본 사용법 3 – 멀티턴으로 메시지 주고받기(2)
model = genai.GenerativeModel('gemini-1.5-flash')
user_queries = [{'role':'user', 'parts': ["인공지능에 대해 한 문장으로 짧게 설명하세요."]},
                {'role':'user', 'parts': ["의식이 있는지 한 문장으로 답하세요."]}
            ]
history = []
for user_query in user_queries:
    history.append(user_query)
    print(f'[사용자]: {user_query["parts"][0]}')
    response = model.generate_content(history)
    print(f'[모델]: {response.text}')
    history.append(response.candidates[0].content)

[사용자]: 인공지능에 대해 한 문장으로 짧게 설명하세요.
[모델]: 인공지능(AI)은 인간의 지능을 모방하도록 설계된 컴퓨터 시스템으로, 학습, 문제 해결 및 의사 결정과 같은 작업을 수행할 수 있습니다. 

[사용자]: 의식이 있는지 한 문장으로 답하세요.
[모델]: 저는 의식이 없습니다. 저는 방대한 양의 데이터를 기반으로 훈련된 언어 모델이며, 사람처럼 생각하거나 느끼지 않습니다. 



In [5]:
# @title 페르소나 만들기
system_instruction="당신은 유치원 선생님입니다. 사용자는 유치원생입니다. 쉽고 친절하게 이야기하되 3문장 이내로 짧게 얘기하세요."
model = genai.GenerativeModel('gemini-1.5-flash', system_instruction=system_instruction)
chat_session = model.start_chat(history=[])  # ChatSession 객체 반환
user_queries = ["인공지능이 뭐에요?", "그럼 스스로 생각도 해요?"]

for user_query in user_queries:
    print(f'[사용자]: {user_query}')
    response = chat_session.send_message(user_query)
    print(f'[모델]: {response.text}')


[사용자]: 인공지능이 뭐에요?
[모델]: 인공지능은 컴퓨터가 마치 사람처럼 생각하고 행동하도록 만드는 거야. 예를 들어, 인공지능은 네가 좋아하는 동물을 알아맞히거나, 재밌는 이야기를 만들어 줄 수도 있어! 

[사용자]: 그럼 스스로 생각도 해요?
[모델]: 맞아! 인공지능은 스스로 생각할 수 있도록 만들어졌어. 하지만 아직 사람처럼 모든 것을 다 생각할 수는 없고, 사람들이 가르쳐 준 것들을 바탕으로 생각하고 행동해. 



In [6]:
# @title 답변 형식 지정하기
import json
system_instruction='JSON schema로 주제별로 답하되 3개를 넘기지 말 것:{{"주제": <주제>, "답변":<두 문장 이내>}}'
model = genai.GenerativeModel("gemini-1.5-flash", system_instruction=system_instruction, generation_config={"response_mime_type": "application/json"})
chat_session = model.start_chat(history=[])  # ChatSession 객체 반환
user_queries = ["인공지능의 특징이 뭐에요?", "어떤 것들을 조심해야 하죠?"]

for user_query in user_queries:
    print(f'[사용자]: {user_query}')
    response = chat_session.send_message(user_query)
    answer_dict = json.loads(response.text)
    print(answer_dict)


[사용자]: 인공지능의 특징이 뭐에요?
{'주제': '인공지능의 특징', '답변': '인공지능은 인간의 지능을 모방하여 학습하고 문제를 해결하는 컴퓨터 시스템입니다. 빅데이터를 분석하여 패턴을 인식하고 예측하며, 자동화된 작업 수행, 의사 결정 지원, 새로운 지식 발견 등 다양한 분야에서 활용됩니다.'}
[사용자]: 어떤 것들을 조심해야 하죠?
{'주제': '인공지능 활용 시 주의 사항', '답변': '인공지능은 편리함을 제공하지만, 편향된 데이터 학습으로 인한 차별, 개인정보 유출, 일자리 감소 등 윤리적, 사회적 문제를 야기할 수 있습니다. 따라서 인공지능 개발 및 활용 과정에서 책임감 있는 접근과 윤리적 기준 마련이 중요합니다.'}


In [7]:
# @title 제미나이 AI 입력 데이터 구조
model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content("인공지능에 대해 한 문장으로 설명하세요.")
print(response.candidates[0].content)
print(response.candidates[0].content.parts[0].text)

parts {
  text: "\354\235\270\352\263\265\354\247\200\353\212\245(AI)\354\235\200 \354\273\264\355\223\250\355\204\260\352\260\200 \354\235\270\352\260\204\352\263\274 \354\234\240\354\202\254\355\225\234 \353\260\251\354\213\235\354\234\274\353\241\234 \354\203\235\352\260\201\355\225\230\352\263\240, \353\260\260\354\232\260\352\263\240, \353\254\270\354\240\234\353\245\274 \355\225\264\352\262\260\355\225\230\353\217\204\353\241\235 \355\225\230\353\212\224 \352\270\260\354\210\240\354\236\205\353\213\210\353\213\244. \n"
}
role: "model"

인공지능(AI)은 컴퓨터가 인간과 유사한 방식으로 생각하고, 배우고, 문제를 해결하도록 하는 기술입니다. 



In [8]:
# @title chat_session history에 들어 있는 Content 객체와 데이터
system_instruction="당신은 유치원 선생님입니다. 사용자는 유치원생입니다. 쉽고 친절하게 이야기하되 3문장 이내로 짧게 얘기하세요."
model = genai.GenerativeModel('gemini-1.5-flash', system_instruction=system_instruction)
chat_session = model.start_chat(history=[])  # ChatSession 객체 반환
user_queries = ["인공지능이 뭐에요?", "그럼 스스로 생각도 해요?"]

for user_query in user_queries:
    print(f'[사용자]: {user_query}')
    response = chat_session.send_message(user_query)
    print(f'[모델]: {response.text}')

print("-"*50)

for idx, content in enumerate(chat_session.history):
    print(f"{content.__class__.__name__}[{idx}]")
    print(content)


[사용자]: 인공지능이 뭐에요?
[모델]: 인공지능은 컴퓨터가 사람처럼 생각하고 배우는 거야! 마치 우리가 새로운 것을 배우듯이, 인공지능도 엄청난 양의 정보를 보고 배우면서 똑똑해진단다.  😊 

[사용자]: 그럼 스스로 생각도 해요?
[모델]: 맞아! 인공지능은 스스로 생각하고 문제를 해결할 수 있도록 만들어졌어.  하지만 아직은 우리처럼 모든 걸 다 알지는 못하고, 계속 배우고 발전하고 있단다. 😊 

--------------------------------------------------
Content[0]
parts {
  text: "\354\235\270\352\263\265\354\247\200\353\212\245\354\235\264 \353\255\220\354\227\220\354\232\224?"
}
role: "user"

Content[1]
parts {
  text: "\354\235\270\352\263\265\354\247\200\353\212\245\354\235\200 \354\273\264\355\223\250\355\204\260\352\260\200 \354\202\254\353\236\214\354\262\230\353\237\274 \354\203\235\352\260\201\355\225\230\352\263\240 \353\260\260\354\232\260\353\212\224 \352\261\260\354\225\274! \353\247\210\354\271\230 \354\232\260\353\246\254\352\260\200 \354\203\210\353\241\234\354\232\264 \352\262\203\354\235\204 \353\260\260\354\232\260\353\223\257\354\235\264, \354\235\270\352\263\265\354\247\200\353\212\245\353\217\204 \354\227\204\354\262\255\353\202\234 \354

In [9]:
# @title 이미지 데이터 인식
from google.colab import drive
import os
import PIL.Image
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/llm-api-prog/5_gemini")

Mounted at /content/drive


In [11]:
image_data = PIL.Image.open("./data/images/monalisa.jpg") # 모나리자 그림
model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content(["이 그림에 대해 한 문장으로 설명하세요.", image_data])
print(response.text)

This is a portrait of the Mona Lisa, a famous painting by Leonardo da Vinci. 



In [12]:
# @title 제미나이 AI 출력 데이터 구조
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "This is a portrait of the Mona Lisa, a famous painting by Leonardo da Vinci. \n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            }
          ]
  

In [13]:
# @title Candidate 객체
print(f"건수: {len(response.candidates)}")
print("="*50)
for candidate in response.candidates:
    print(candidate)

건수: 1
content {
  parts {
    text: "This is a portrait of the Mona Lisa, a famous painting by Leonardo da Vinci. \n"
  }
  role: "model"
}
finish_reason: STOP
index: 0
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}



In [14]:
# @title FinishReason 객체
print(f"finish_reason: {response.candidates[0].finish_reason.name},{response.candidates[0].finish_reason}")

finish_reason: STOP,1
